# Readme
1. This is the code for predicting aesthetic scores or histograms using the method in the paper:
Jin, Bin, Maria V. Ortiz Segovia, and Sabine Süsstrunk. "Image aesthetic predictors based on weighted CNNs." Image Processing (ICIP), 2016 IEEE International Conference on. IEEE, 2016.

2. To use the code, you need to install the following packages:
Theano(>=0.9), Keras(>=1.0), numpy, opencv

3. To run the code: please download the weights from our project webpage.
There are two weights hdf5 files, one is for predicting the aesthetics score (1 value), the other one is for predicting the histogram (10 bins). Both of them are ~500 MB since the network architecture is VGG16.
Modify the "outputflag" parameter to change between score prediction or histogram prediction. 

----------Created by Bin Jin. Nov. 16th, 2016.

## Load the packages

In [ ]:
from __future__ import division

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2, os
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2


## Define the network architecture

In [ ]:
def AestheticNet(outputflag='score',weight_decay=0.0005, weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2))) # The receptive filed is 7*7 here

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))

    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu',init='he_normal',W_regularizer=l2(weight_decay)))

    model.add(Dropout(0.5))
    if outputflag =='score' :
        model.add(Dense(1,init='he_normal',W_regularizer=l2(weight_decay)))
    else:
        model.add(Dense(10, activation='softmax',init='he_normal',W_regularizer=l2(weight_decay)))    

    if weights_path:
        model.load_weights(weights_path)

    return model

## Load the weights of the network

In [ ]:
outputflag = 'score' ## 'score' for predicting the aesthetic score (1 value),'hist' for predicting the histogram (10 bins)
path_weights = './' ## modify to the path where you put the downloaded weights
weights_file = 'AVA_AestheticNet_' + outputflag + '_ICIP.h5'
model = AestheticNet(outputflag=outputflag,weight_decay=0.0005, weights_path=path_weights + weights_file)

## Prediction

In [ ]:
path_images = './'
filename = 'example.jpg' ## specify your own image

img = cv2.imread(path_images + filename)
imgn = np.copy(img)
imgn = cv2.resize(imgn,(224,224))
imgn = np.transpose(imgn,[2,0,1])
imgn = np.expand_dims(imgn,axis=0)
score = model.predict(imgn)

img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

print filename + ': ', score
